# Extract MODIS data
- using 2km scale and WGS84 projection
- For entire CONUS
- images with data for 3 months

In [1]:
import initialise
import common
import ee
ee.Initialize()

In [2]:
PROJ = common.MAP_PROJ
SCALE = common.MAP_SCALE
USA = ee.FeatureCollection('USDOS/LSIB/2017').filter("COUNTRY_NA == 'United States'")
FLOAT_PRE = 5
# START_DATES = ["2016-10-01", "2017-01-01", "2017-04-01", "2017-07-01"]
# END_DATES = ["2017-01-01", "2017-04-01", "2017-07-01", "2017-10-01"]
START_DATES = ["2016-04-01", "2016-07-01"]
END_DATES = ["2016-07-01", "2016-10-01"]
bbox = ee.Geometry.BBox(*common.MAP_BOUNDS)
conus_bbox = bbox.intersection(USA.geometry(), 1, PROJ)
FOLDER = f"GEE_{PROJ.replace(':', '-')}_{int(SCALE)}"

In [3]:
# MODIS data details
PRODUCT = "MODIS/006/MCD43A4"
BANDS = ["Nadir_Reflectance_Band1",
         "Nadir_Reflectance_Band2",
         "Nadir_Reflectance_Band3",
         "Nadir_Reflectance_Band4",
         "Nadir_Reflectance_Band5",
         "Nadir_Reflectance_Band6",
         "Nadir_Reflectance_Band7"]
NEW_BANDS = ["Band1", "Band2", "Band3", "Band4", "Band5", "Band6", "Band7", ]

In [4]:
images = []
for num in range(len(START_DATES)):
    print(START_DATES[num], END_DATES[num])
    modis_col = ee.ImageCollection(PRODUCT).select(BANDS, NEW_BANDS).filterDate(START_DATES[num], END_DATES[num])
    images.append(modis_col.toBands().clip(conus_bbox).regexpRename('^', 'b'))

2016-04-01 2016-07-01
2016-07-01 2016-10-01


In [5]:
for num, image in enumerate(images):
    task = ee.batch.Export.image.toDrive(
        image=image,
        description=f'MODIS_{START_DATES[num]}',
        folder=FOLDER,
        fileFormat='GeoTIFF',
        scale=SCALE,
        crs=PROJ,
        skipEmptyTiles=True
    );
    task.start()